https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

In [1]:
import glob
import pandas as pd
import numpy as np
import json

In [2]:
annotations=glob.glob('data/OPP-115/annotations/*')

In [3]:
annotations_list=[]
for file in annotations:
    annotations_list.append(pd.read_csv(file, header=None))

In [4]:
annot=pd.concat(annotations_list, ignore_index=True)

In [5]:
annot.columns = ['annotation_ID', 'batch_ID', 'annotator_ID', 'policy_ID', 'segment_ID','category_name',
            'attribute_value_pairs','date','policy_URL']

In [6]:
annot['dict'] = annot.attribute_value_pairs.apply(lambda x: json.loads(x))

In [7]:
annot.head()

,annotation_ID,batch_ID,annotator_ID,policy_ID,segment_ID,category_name,attribute_value_pairs,date,policy_URL,dict
0,13160,test_category_labeling_highlight_fordham_aaaa,121,3828,0,Other,"{""Other Type"": {""endIndexInSegment"": 575, ""sta...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Other Type': {'endIndexInSegment': 575, 'sta..."
1,13161,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
2,13162,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
3,13163,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
4,13164,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."


In [8]:
for x in annot['dict']:
    for key, value in x.items():
        value.pop('endIndexInSegment', None)
        value.pop('startIndexInSegment', None)

In [9]:
text=[]
category=[]
subcat=[]
label=[]
counter=0
for i,x in enumerate(annot['dict']):
    for key, value in x.items():
        subcat.append(key)
        if value.get('selectedText')==None:
            text.append('noSelectedText')
        else:
            text.append(value.get('selectedText'))  
        category.append(annot['category_name'][i])
        for k, v in value.items():
            if k=='value':
                label.append(v)

In [10]:
d=list(zip(text, category, subcat, label))

In [11]:
data=pd.DataFrame(d, columns=['text', 'category', 'subcategory', 'label'])

In [12]:
data.head()

,text,category,subcategory,label
0,"Effective Date: May 7, 2015 Kraft Site Privacy...",Other,Other Type,Introductory/Generic
1,noSelectedText,First Party Collection/Use,Collection Mode,not-selected
2,collec,First Party Collection/Use,Choice Scope,Collection
3,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website
4,"personally-identifiable information, such as",First Party Collection/Use,Personal Information Type,Generic personal information


In [13]:
data['cat_sub'] = data['category'] +'-'+ data['subcategory']

In [14]:
data['cat_sub_lab'] = data['category'] +'-'+ data['subcategory'] +'-'+ data['label']

In [15]:
import string, re
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct
data['text'] = data['text'].apply(lambda x : remove_punct(x.lower()))

data = data[data['text']!='null']
data = data[data['text']!='noselectedtext']
data = data[data['text']!='']
data = data[data['text']!=' ']
data.reset_index(inplace=True, drop=True)
ds = data.drop_duplicates()

In [16]:
ds.shape

(36717, 6)

In [17]:
ds.groupby('category').count()

,text,subcategory,label,cat_sub,cat_sub_lab
category,,,,,
Data Retention,676,676,676,676,676
Data Security,808,808,808,808,808
Do Not Track,67,67,67,67,67
First Party Collection/Use,15346,15346,15346,15346,15346
International and Specific Audiences,582,582,582,582,582
Other,2759,2759,2759,2759,2759
Policy Change,932,932,932,932,932
Third Party Sharing/Collection,10655,10655,10655,10655,10655
"User Access, Edit and Deletion",1094,1094,1094,1094,1094


In [18]:
ds.groupby(['category','subcategory']).count().shape

(36, 4)

In [19]:
ds.groupby(['category','subcategory','label']).count().shape

(259, 3)

In [20]:
ds['tokenized'] = ds['text'].apply(lambda x : re.split(' ', x))
# ds['tokenized_text']=res

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
from gensim.models import Word2Vec
import logging
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [22]:
w2v = Word2Vec.load('word2vec.model')

2019-09-20 18:00:46,637 : INFO : loading Word2Vec object from word2vec.model
/anaconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-09-20 18:00:46,678 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2019-09-20 18:00:46,678 : INFO : setting ignored attribute vectors_norm to None
2019-09-20 18:00:46,679 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2019-09-20 18:00:46,680 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2019-09-20 18:00:46,684 : INFO : setting ignored attribute cum_table to None
2019-09-20 18:00:46,691 : INFO : loaded word2vec.model


In [23]:
from sklearn.preprocessing import OneHotEncoder
# encode = OneHotEncoder()

In [24]:
ds.reset_index(inplace=True, drop=True)

In [25]:
word_vectors = w2v.wv

In [26]:
word2id = {k: v.index for k, v in word_vectors.vocab.items()}

In [27]:
def add_new_word(new_word, new_vector, new_index, embedding_matrix, word2id):
    embedding_matrix = np.insert(embedding_matrix, [new_index], [new_vector], axis=0)
    
    word2id = {word: (index+1) if index >= new_index else index for word, index in word2id.items()}
    word2id[new_word] = new_index
    return embedding_matrix, word2id

UNK_INDEX = 0
UNK_TOKEN = 'UNK'

embedding_matrix = word_vectors.vectors
unk_vector = embedding_matrix.mean(0)
embedding_matrix, word2id = add_new_word(UNK_TOKEN, unk_vector, UNK_INDEX, embedding_matrix, word2id)

In [28]:
embedding_matrix = np.insert(embedding_matrix, len(embedding_matrix), [np.zeros((100,))], axis=0)

In [29]:
def get_int_data(token_text, word2id):
    x = []
    unk_count = 0
    for item in token_text:
        temp=[]
        x.append(temp)
        for word in item:
            if word in word2id:
                temp.append(word2id.get(word))
            else:
                temp.append(UNK_INDEX)
                unk_count += 1
    print('Data created. Percentage of unknown words: %.3f' % (unk_count))
    return np.array(x)

x=get_int_data(ds.tokenized, word2id)
x.shape

Data created. Percentage of unknown words: 7006.000


(36717,)

In [30]:
ds['enumerated_text']=x

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
ds.head()

,text,category,subcategory,label,cat_sub,cat_sub_lab,tokenized,enumerated_text
0,effective date may 7 2015 kraft site privacy n...,Other,Other Type,Introductory/Generic,Other-Other Type,Other-Other Type-Introductory/Generic,"[effective, date, may, 7, 2015, kraft, site, p...","[553, 244, 11, 2150, 839, 1332, 35, 26, 139, 2..."
1,collec,First Party Collection/Use,Choice Scope,Collection,First Party Collection/Use-Choice Scope,First Party Collection/Use-Choice Scope-Collec...,[collec],[2151]
2,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website,First Party Collection/Use-Action First-Party,First Party Collection/Use-Action First-Party-...,"[register, on, our, website, or, participate, ...","[204, 19, 10, 52, 6, 212, 13, 10, 168, 3, 363]"
3,personallyidentifiable information such as,First Party Collection/Use,Personal Information Type,Generic personal information,First Party Collection/Use-Personal Informatio...,First Party Collection/Use-Personal Informatio...,"[personallyidentifiable, information, such, as]","[630, 5, 31, 23]"
4,if you choose to register on our website or pa...,First Party Collection/Use,Choice Type,Opt-in,First Party Collection/Use-Choice Type,First Party Collection/Use-Choice Type-Opt-in,"[if, you, choose, to, register, on, our, websi...","[32, 4, 105, 1, 204, 19, 10, 52, 6, 212, 13, 1..."


In [32]:
data_retention = ds[ds.category=='Data Retention']
data_retention['Personal_Information_Type'] = data_retention['subcategory'].apply(lambda x: 1 if x=='Personal Information Type' else 0)
data_retention['Retention_Period'] = data_retention['subcategory'].apply(lambda x: 1 if x=='Retention Period' else 0)
data_retention['Retention_Purpose'] = data_retention['subcategory'].apply(lambda x: 1 if x=='Retention Purpose' else 0)

first_party_collection_use = ds[ds.category=='First Party Collection/Use']
first_party_collection_use['Action_First_Party'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Action First-Party' else 0)
first_party_collection_use['Choice_Scope'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Choice Scope' else 0)
first_party_collection_use['Choice_Type'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Choice Type' else 0)
first_party_collection_use['Collection_Mode'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Collection Mode' else 0)
first_party_collection_use['Does_Does_Not'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Does/Does Not' else 0)
first_party_collection_use['Identifiability'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Identifiability' else 0)
first_party_collection_use['Personal_Information_Type'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Personal Information Type' else 0)
first_party_collection_use['Purpose'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='Purpose' else 0)
first_party_collection_use['User_Type'] = first_party_collection_use['subcategory'].apply(lambda x: 1 if x=='User Type' else 0)

policy_change = ds[ds.category=='Policy Change']
policy_change['Change_Type'] = policy_change['subcategory'].apply(lambda x: 1 if x=='Change Type' else 0)
policy_change['Notification_Type'] = policy_change['subcategory'].apply(lambda x: 1 if x=='Notification Type' else 0)
policy_change['User_Choice'] = policy_change['subcategory'].apply(lambda x: 1 if x=='User Choice' else 0)

third_party_sharing_collection = ds[ds.category=='Third Party Sharing/Collection']
third_party_sharing_collection['Action_Third_Party'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Action Third Party' else 0)
third_party_sharing_collection['Choice_Scope'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Choice Scope' else 0)
third_party_sharing_collection['Choice_Type'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Choice Type' else 0)
third_party_sharing_collection['Does_Does_Not'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Does/Does Not' else 0)
third_party_sharing_collection['Identifiability'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Identifiability' else 0)
third_party_sharing_collection['Personal_Information_Type'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Personal Information Type' else 0)
third_party_sharing_collection['Purpose'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Purpose' else 0)
third_party_sharing_collection['Third_Party_Entity'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='Third Party Entity' else 0)
third_party_sharing_collection['User_Type'] = third_party_sharing_collection['subcategory'].apply(lambda x: 1 if x=='User Type' else 0)

user_access_edit_deletion = ds[ds.category=='User Access, Edit and Deletion']
user_access_edit_deletion['Access_Scope'] = user_access_edit_deletion['subcategory'].apply(lambda x: 1 if x=='Access Scope' else 0)
user_access_edit_deletion['Access_Type'] = user_access_edit_deletion['subcategory'].apply(lambda x: 1 if x=='Access Type' else 0)
user_access_edit_deletion['User_Type'] = user_access_edit_deletion['subcategory'].apply(lambda x: 1 if x=='User Type' else 0)

user_choice_control = ds[ds.category=='User Choice/Control']
user_choice_control['Choice_Scope'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='Choice Scope' else 0)
user_choice_control['Choice_Type'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='Choice Type' else 0)
user_choice_control['Personal_Information_Type'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='Personal Information Type' else 0)
user_choice_control['Purpose'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='Purpose' else 0)
user_choice_control['User_Type'] = user_choice_control['subcategory'].apply(lambda x: 1 if x=='User Type' else 0)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [33]:
#first_party_collection_use.sum()

In [34]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Flatten, Dropout
from keras import regularizers
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sklearn.model_selection import train_test_split
from keras.models import load_model
max_length = max(ds.enumerated_text.apply(lambda x: len(x)))

Using TensorFlow backend.


DATA RETENTION MODELS

In [35]:
# Personal Information Type Model (Data Retention)

padded_docs = pad_sequences(data_retention.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, data_retention.Personal_Information_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 3.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('data_retention_personal_information_type.h5')
model = load_model('data_retention_personal_information_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

2019-09-20 18:00:53,736 : WARNING : From /anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
2019-09-20 18:00:54,040 : WARNING : From /anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


[[325  18]
 [  1 129]]
0.9314079422382673
0.9923076923076923
[[122  23]
 [  6  52]]
0.7819548872180451
0.896551724137931


In [36]:
# Retention Period Model (Data Retention)

padded_docs = pad_sequences(data_retention.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, data_retention.Retention_Period, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('data_retention_rentention_period.h5')

model=load_model('data_retention_rentention_period.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[242  69]
 [  3 159]]
0.8153846153846153
0.9814814814814815
[[90 41]
 [15 57]]
0.6705882352941176
0.7916666666666666


In [37]:
# Retention Purpose Model (Data Retention)

padded_docs = pad_sequences(data_retention.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, data_retention.Retention_Purpose, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 10.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dropout(0.2))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))


# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('data_retention_retention_purpose.h5')
model=load_model('data_retention_retention_purpose.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

2019-09-20 18:00:57,250 : WARNING : From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[[150 142]
 [  0 181]]
0.7182539682539683
1.0
[[50 80]
 [ 8 65]]
0.5963302752293578
0.8904109589041096


FIRST PARTY COLLECTION/USE

In [38]:
# Action First-Party Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Action_First_Party, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 20.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_use_action_first_party.h5')
model=load_model('first_party_collection_use_action_first_party.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[5103 3698]
 [  12 1929]]
0.5097780126849895
0.9938176197836167
[[1971 1841]
 [ 111  681]]
0.41098370549185276
0.8598484848484849


In [39]:
# Choice Scope Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Choice_Scope, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 75.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dropout(0.2))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_use_choice_scope.h5')

model = load_model('first_party_collection_use_choice_scope.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[7130 3263]
 [   0  349]]
0.1762181267356728
1.0
[[2843 1582]
 [  46  133]]
0.14044350580781415
0.7430167597765364


In [40]:
# Choice Type Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Choice_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 50.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_use_choice_type.h5')
model = load_model('first_party_collection_use_choice_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[8076 2037]
 [   0  629]]
0.3817905918057663
1.0
[[3258 1038]
 [  46  262]]
0.32587064676616917
0.8506493506493507


In [41]:
# Collection Mode Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Collection_Mode, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 50.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dropout(0.2))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_collection_mode.h5')
model = load_model('first_party_collection_collection_mode.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[5134 4565]
 [   5 1038]]
0.31236834185976525
0.9952061361457335
[[1952 2167]
 [  81  404]]
0.2643979057591623
0.8329896907216495


In [42]:
# Does/Does Not Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Does_Does_Not, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 50.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dropout(0.2))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_use_does_does_not.h5')
model = load_model('first_party_collection_use_does_does_not.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[6034 3787]
 [   5  916]]
0.32574679943101
0.99457111834962
[[2408 1809]
 [  42  345]]
0.2715466351829988
0.8914728682170543


In [43]:
# Identifiability Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Identifiability, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 75.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_use_identifiability.h5')
model = load_model('first_party_collection_use_identifiability.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[8821 1466]
 [   0  455]]
0.382996632996633
1.0
[[3642  781]
 [  43  138]]
0.2509090909090909
0.7624309392265194


In [44]:
# Personal Information Type Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Personal_Information_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_use_personal_information_type.h5')
model = load_model('first_party_collection_use_personal_information_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[7712  803]
 [  39 2188]]
0.8386354925258719
0.9824876515491693
[[3176  536]
 [ 184  708]]
0.6629213483146067
0.7937219730941704


In [45]:
# Purpose Model (First Party Collection/Use)
from keras.models import load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix, f1_score
from sklearn import preprocessing
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Flatten

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.Purpose, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_use_purpose.h5')

model = load_model('first_party_collection_use_purpose.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[7156  648]
 [  55 2883]]
0.8913278713866132
0.9812797821647379
[[2842  486]
 [ 225 1051]]
0.7472449342339139
0.8236677115987461


In [46]:
# User Type Model (First Party Collection/Use)

padded_docs = pad_sequences(first_party_collection_use.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, first_party_collection_use.User_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 50.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_use_user_type.h5')
model = load_model('first_party_collection_use_user_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[9748  755]
 [   3  236]]
0.383739837398374
0.9874476987447699
[[4079  421]
 [  28   76]]
0.2529118136439268
0.7307692307692307


POLICY CHANGE

In [47]:
# Change Type Model (Policy Change)

padded_docs = pad_sequences(policy_change.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, policy_change.Change_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('policy_change_change_type.h5')
model = load_model('policy_change_change_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[318  90]
 [  3 241]]
0.8382608695652175
0.9877049180327869
[[120  63]
 [ 26  71]]
0.6147186147186147
0.7319587628865979


In [48]:
# Notification Type Model (Policy Change)

padded_docs = pad_sequences(policy_change.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, policy_change.Notification_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('policy_change_notification_type.h5')

model = load_model('policy_change_notification_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[273 113]
 [  0 266]]
0.8248062015503876
1.0
[[ 73  70]
 [ 26 111]]
0.6981132075471699
0.8102189781021898


In [49]:
# User Choice Model (Policy Change)

padded_docs = pad_sequences(policy_change.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, policy_change.User_Choice, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 20.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('policy_change_user_choice.h5')

model = load_model('policy_change_user_choice.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[365 145]
 [  0 142]]
0.662004662004662
1.0
[[128 106]
 [ 12  34]]
0.3655913978494624
0.7391304347826086


THIRD PARTY SHARING/COLLECTION

In [50]:
# Action Third-Party Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Action_Third_Party, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 20.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection_action_third_party.h5')
model = load_model('third_party_sharing_collection_action_third_party.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[4548 1602]
 [   7 1301]]
0.6179054856328663
0.9946483180428135
[[1784  832]
 [ 121  460]]
0.4911906033101975
0.7917383820998278


In [51]:
# Choice Scope Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Choice_Scope, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 75.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection_choice_scope.h5')
model = load_model('third_party_sharing_collection_choice_scope.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[5075 2140]
 [   1  242]]
0.18438095238095234
0.9958847736625515
[[2142  956]
 [  22   77]]
0.13604240282685512
0.7777777777777778


In [52]:
# Choice Type Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Choice_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 10.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection_choice_type.h5')
model = load_model('third_party_sharing_collection_choice_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[6804  185]
 [   6  463]]
0.829006266786034
0.9872068230277186
[[2866  119]
 [  57  155]]
0.6378600823045268
0.7311320754716981


In [53]:
# Does/Does Not Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Does_Does_Not, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 10.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection_does_does_not.h5')
model = load_model('third_party_sharing_collection_does_does_not.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[4789 2010]
 [  30  629]]
0.3814432989690722
0.9544764795144158
[[1987  932]
 [  60  218]]
0.30532212885154064
0.7841726618705036


In [54]:
# Identifiability Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Identifiability, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 25.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection_identifiability.h5')
model = load_model('third_party_sharing_collection_identifiability.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[6037 1147]
 [   4  270]]
0.31933767001774105
0.9854014598540146
[[2559  526]
 [  25   87]]
0.24000000000000002
0.7767857142857143


In [55]:
# Personal Information Type Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Personal_Information_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection_personal_information_type.h5')
model = load_model('third_party_sharing_collection_personal_information_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[5950  478]
 [  24 1006]]
0.8003182179793159
0.9766990291262136
[[2484  280]
 [  71  362]]
0.6734883720930233
0.836027713625866


In [56]:
# Purpose Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Purpose, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection_purpose.h5')
model = load_model('third_party_sharing_collection_purpose.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[5087  451]
 [  46 1874]]
0.8829210836277974
0.9760416666666667
[[1997  393]
 [ 145  662]]
0.7110633727175081
0.8203221809169765


In [57]:
# Third Part Entity Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.Third_Party_Entity, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection_third_party_entity.h5')
model = load_model('third_party_sharing_collection_third_party_entity.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[5501  503]
 [  70 1384]]
0.8284944627357079
0.951856946354883
[[2194  363]
 [ 165  475]]
0.6427604871447903
0.7421875


In [58]:
# User Type Model (Third Party Sharing/Collection)

padded_docs = pad_sequences(third_party_sharing_collection.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

# x_train, x_test, y_train, y_test = train_test_split(padded_docs, third_party_sharing_collection.User_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 40.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection_user_type.h5')
model = load_model('third_party_sharing_collection_user_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[5544  460]
 [1399   55]]
0.055865921787709494
0.03782668500687758
[[2342  215]
 [ 613   27]]
0.061224489795918366
0.0421875


USER ACCESS, EDIT, AND DELETION

In [59]:
# Access Scope Model (User Access, Edit and Deletion)

padded_docs = pad_sequences(user_access_edit_deletion.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_access_edit_deletion.Access_Scope, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_access_edit_deletion_access_scope.h5')
model = load_model('user_access_edit_deletion_access_scope.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[375  96]
 [ 13 281]]
0.8375558867362145
0.95578231292517
[[135  67]
 [ 35  92]]
0.6433566433566434
0.7244094488188977


In [60]:
# Access Type Model (User Access, Edit and Deletion)

padded_docs = pad_sequences(user_access_edit_deletion.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_access_edit_deletion.Access_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_access_edit_deletion_access_type.h5')
model = load_model('user_access_edit_deletion_access_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[304  98]
 [  4 359]]
0.875609756097561
0.9889807162534435
[[114  64]
 [ 27 124]]
0.7315634218289085
0.8211920529801324


In [61]:
# User Type Model (User Access, Edit and Deletion)

padded_docs = pad_sequences(user_access_edit_deletion.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_access_edit_deletion.User_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_access_edit_deletion_user_type.h5')
model = load_model('user_access_edit_deletion_user_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[585  72]
 [  5 103]]
0.7279151943462898
0.9537037037037037
[[243  35]
 [ 11  40]]
0.634920634920635
0.7843137254901961


USER CHOICE/CONTROL

In [62]:
# Choice Scope Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.Choice_Scope, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 10.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_choice_control_choice_scope.h5')
model = load_model('user_choice_control_choice_scope.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[1027  876]
 [   7  748]]
0.6288356452290879
0.990728476821192
[[288 508]
 [ 61 283]]
0.49867841409691627
0.8226744186046512


In [63]:
# Choice Type Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.Choice_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 5.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_choice_control_choice_type.h5')
model = load_model('user_choice_control_choice_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[1514  249]
 [   6  889]]
0.8745696015740285
0.9932960893854749
[[574 199]
 [ 68 299]]
0.6913294797687862
0.8147138964577657


In [64]:
# Personal Information Type Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.Personal_Information_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 25.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_choice_control_personal_information_type.h5')
model = load_model('user_choice_control_personal_information_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[1736  580]
 [   0  342]]
0.5411392405063291
1.0
[[648 342]
 [ 38 112]]
0.3708609271523179
0.7466666666666667


In [65]:
# Purpose Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.Purpose, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 15.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_choice_control_purpose.h5')
model = load_model('user_choice_control_purpose.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[1231  852]
 [   3  572]]
0.5722861430715358
0.9947826086956522
[[435 478]
 [ 43 184]]
0.41394825646794153
0.8105726872246696


In [66]:
# User Type Model (User Choice/Control)

padded_docs = pad_sequences(user_choice_control.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, user_choice_control.User_Type, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 15.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_choice_control_user_type.h5')
model = load_model('user_choice_control_user_type.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[2453  114]
 [   1   90]]
0.6101694915254238
0.989010989010989
[[1029   59]
 [  15   37]]
0.5
0.7115384615384616
